##**The Language Model Protein Folder**


Noa Klugman, Yaad Luria, Elad Applbaum, Eyal Zur

**About The Project**

words

#Installs and Imports

In [1]:
%%time

!pip install import-ipynb
import import_ipynb
import os, time


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.3 MB/s eta 0:00:00
CPU times: user 188 ms, sys: 12.4 ms, total: 201 ms
Wall time: 8.6 s


In [2]:
from google.colab import files
import re
import os
import tensorflow as tf
from tensorflow import keras

In [3]:
from google.colab import files
import re
import os
from IPython.display import clear_output

In [4]:
#@title Clone NanoFoldLM trained model


%%bash
# rm -rf NanoFoldLM
# git clone https://github.com/YaadLuria/NanoFoldLM --quiet

if [ ! -f NanoFoldLMReady ]; then
  # install dependencies
  pip -q install biopython
  pip install py3Dmol
  pip install pdb-tools


  # download model
  if [ ! -d "NanoNet/" ]; then
    git clone https://github.com/YaadLuria/NanoFoldLM --quiet
  fi
  touch NanoFoldLMReady
fi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 27.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 5.3 MB/s eta 0:00:00


In [5]:
# %% [code]
#@title Test git use
os.chdir("/content/NanoFoldLM/NanoFoldLM")
import utils
# if "NanoFoldLM" not in dir():
#   from timeit import default_timer as timer
#   import sys
#   sys.path.insert(0, '/content/NanoFoldLM/')
#   from NanoFoldLM import *

print(utils.NB_MAX_LENGTH) # just for git clone check
import py3Dmol

importing Jupyter notebook from utils.ipynb
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.4/276.4 kB 5.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
140


#Set Parameters

In [6]:
jobname = "7DJY" #@param {type:"string"}
jobname = re.sub(r'\W+', '', jobname)[:50]


download_path = "/drive/MyDrive/Hackathon3D" #@param {type:"string"}

sequence = "QVQLVESGGGLVQAGGSLRLSCAASGATFITYGMTWFRQAPGKEREFVAAVTGNGAGTTYLPSVKGRFTISRDNAKNTVYLQMSSLKPEDTAVYYCGGRRWVPATAVDQVAYWGQGTQVTVSSHHHHHH" #@param {type:"string"}
sequence = re.sub("[^A-Z:]", "", sequence.replace("/",":").upper())
sequence = re.sub(":+",":",sequence)
sequence = re.sub("^[:]+","",sequence)
sequence = re.sub("[:]+$","",sequence)



amino_acids = set("ACDEFGHIKLMNPQRSTVWY")
for i,aa in enumerate(sequence):
  if aa.upper() not in amino_acids:
    print(f"Invalid amino acid: {aa} in position {i}")

# if utils.check_valid_protein_sequence(sequence):
#   print(f"Invalid amino acid: {aa}")

seqs = sequence.split(":")
lengths = [len(s) for s in seqs]
length = sum(lengths)



download_pdb = True #@param {type:"boolean"}
print_atom_coordinates = False #@param {type:"boolean"}
embeding_size = "320" #@param ["320", "480", "640",  "1280", "One_hot_encoding"] {type:"string"}




Compare to solved pdb file

In [8]:
have_solved_structure = True #@param {type:"boolean"}
pdb_id = "6XW6" #@param {type:"string"}
chain_id = "C" #@param {type:"string"}
if have_solved_structure:
  seq, atoms = utils.get_seq_aa_by_id(pdb_id, chain_id)


Structure exists: './pdb6xw6.ent' 


AttributeError: ignored

#Run model

In [ ]:
if embeding_size ==  "One_hot_encoding":
  seq_embeddings = utils.generate_input_one_hot(sequence)
else:
  seq_embeddings = utils.get_esm_embedding_for_protein_sequence(sequence, int(embeding_size))

model_path = f"./Models"
model_dict = {"320": "model_320", "480": "model_480", "640": "model_640",  "1280": "model_1280", "One_hot_encoding": "model_22"}
model = tf.keras.models.load_model(f'{model_path}/{model_dict[embeding_size]}.tf')
predict = model.predict(seq_embeddings)
utils.matrix_to_pdb(sequence, predict[0], jobname)
# ATOM_LINE = "ATOM{}{}  {}{}{} {}{}{}{}{:.3f}{}{:.3f}{}{:.3f}  1.00{}{:.2f}           {}\n"
# END_LINE = "END\n"
# BACKBONE_ATOMS = ["N", "CA", "C", "O", "CB"]

# k = 1
# coord_matrix = predict[0]
# with open(f"{jobname}.pdb", "w") as pdb_file:
#     for i, aa in enumerate(sequence):
#         third_space = (4 - len(str(i))) * " "
#         for j, atom in enumerate(BACKBONE_ATOMS):
#             if not (aa == "G" and atom == "CB"):  # GLY lacks CB atom
#                 x, y, z = coord_matrix[i][3*j], coord_matrix[i][3*j+1], coord_matrix[i][3*j+2]
#                 b_factor = 0.00
#                 first_space = (7 - len(str(k))) * " "
#                 second_space = (4 - len(atom)) * " "
#                 forth_space = (12 - len("{:.3f}".format(x))) * " "
#                 fifth_space = (8 - len("{:.3f}".format(y))) * " "
#                 sixth_space = (8 - len("{:.3f}".format(z))) * " "
#                 seventh_space = (6 - len("{:.2f}".format(b_factor))) * " "

#                 pdb_file.write(ATOM_LINE.format(first_space, k, atom, second_space, Polypeptide.one_to_three(aa) , "H", third_space,
#                                                 i, forth_space, x, fifth_space, y, sixth_space, z, seventh_space,
#                                                 b_factor, atom[0]))
#                 k += 1

#     pdb_file.write(END_LINE)


In [ ]:

#@title display (optional) {run: "auto"}
pdb_predict = f"{jobname}.pdb"
# pdb_file_path =  "/content/drive/MyDrive/Hackathon3D/Datasets/Train_set/12E8_1.pdb"

with open(pdb_predict, 'r') as pdb_predicts:
  pdb_str = pdb_predicts.read()


from string import ascii_uppercase, ascii_lowercase
alphabet_list = list(ascii_uppercase+ascii_lowercase)
pymol_color_list = ["#33ff33","#00ffff","#ff33cc","#ffff00","#ff9999","#e5e5e5","#7f7fff","#ff7f00",
                    "#7fff7f","#199999","#ff007f","#ffdd5e","#8c3f99","#b2b2b2","#007fff","#c4b200",
                    "#8cb266","#00bfbf","#b27f7f","#fcd1a5","#ff7f7f","#ffbfdd","#7fffff","#ffff7f",
                    "#00ff7f","#337fcc","#d8337f","#bfff3f","#ff7fff","#d8d8ff","#3fffbf","#b78c4c",
                    "#339933","#66b2b2","#ba8c84","#84bf00","#b24c66","#7f7f7f","#3f3fa5","#a5512b"]





def show_pdb(pdb_str, show_sidechains=False, show_mainchains=False,
             color="pLDDT", chains=None, vmin=50, vmax=90,
             size=(800,480), hbondCutoff=4.0,
             Ls=None,
             animate=False):

  if chains is None:
    chains = 1 if Ls is None else len(Ls)
  view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js', width=size[0], height=size[1])

  if animate:
    view.addModelsAsFrames(pdb_str,'pdb',{'hbondCutoff':hbondCutoff})
  else:
    view.addModel(pdb_str,'pdb',{'hbondCutoff':hbondCutoff})
  if color == "pLDDT":
    view.setStyle({'cartoon': {'colorscheme': {'prop':'b','gradient': 'roygb','min':vmin,'max':vmax}}})
  elif color == "rainbow":
    view.setStyle({'cartoon': {'color':'spectrum'}})
  elif color == "chain":
    for n,chain,color in zip(range(chains),alphabet_list,pymol_color_list):
       view.setStyle({'chain':chain},{'cartoon': {'color':color}})
  if show_sidechains:
    BB = ['C','O','N']
    view.addStyle({'and':[{'resn':["GLY","PRO"],'invert':True},{'atom':BB,'invert':True}]},
                  {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
    view.addStyle({'and':[{'resn':"GLY"},{'atom':'CA'}]},
                  {'sphere':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
    view.addStyle({'and':[{'resn':"PRO"},{'atom':['C','O'],'invert':True}]},
                  {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
  if show_mainchains:
    BB = ['C','O','N','CA']
    view.addStyle({'atom':BB},{'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
  view.zoomTo()
  if animate: view.animate()
  return view

color = "rainbow" #@param ["confidence", "rainbow", "chain"]
if color == "confidence": color = "pLDDT"
show_sidechains = True #@param {type:"boolean"}
show_mainchains = True #@param {type:"boolean"}
show_pdb(pdb_str, color=color,
         show_sidechains=show_sidechains,
         show_mainchains=show_mainchains,
         Ls=lengths).show()
if(have_solved_structure):

# Read the specific chain

  print("********pdb_test********")
  with open('test.pdb', 'r') as pdb_:
    pdb_test = pdb_.read()
  print(pdb_test)

  show_pdb(atoms, color=color,
        show_sidechains=show_sidechains,
        show_mainchains=show_mainchains,
        Ls=lengths).show()

In [ ]:
#@title Download predicted pdb (optional)
if download_pdb:
  from google.colab import files
  files.download(f"{jobname}.pdb")





In [ ]:
# from google.colab import drive
# drive.mount('/content/drive/')
# %cd "/content/drive/MyDrive/Hackathon3D"
# import utils